In [22]:
import psycopg2
from psycopg2 import Error
import glob, os
import random
from time import sleep
from bs4 import BeautifulSoup
import requests

user="postgres"
password="adminYasser"
host="database-2.cxv32gki8sxt.us-east-1.rds.amazonaws.com"
port="5432"
database="imdb"

################################################
###### First time to create the database #######
################################################

# try:
#     # Connect to an existing database
#     connection = psycopg2.connect(user=user,
#                                   password=password,
#                                   host=host,
#                                   port=port,
#                                   database=database)

#     # Create a cursor to perform database operations
#     cursor = connection.cursor()
#     # Print PostgreSQL details
#     print("PostgreSQL server information")
#     print(connection.get_dsn_parameters(), "\n")
#     # Executing a SQL query
#     cursor.execute("SELECT version();")
#     # Fetch result
#     record = cursor.fetchone()
#     print("You are connected to - ", record, "\n")
    
#     create_table_query0 = "DROP TABLE IF EXISTS reviews;"
#     create_table_query1 = "DROP TABLE IF EXISTS movies;"
    
#     create_table_query2 = '''CREATE TABLE movies
#             (
#                 "id" character varying(45) NOT NULL,
#                 title text NOT NULL,
#                 "url" text NOT NULL,
#                 PRIMARY KEY ("id")
#             ); '''
    
    
#     create_table_query3 = '''CREATE TABLE reviews
#             (
#                 "id" serial,
#                 review_id character varying(45),
#                 review text NOT NULL,
#                 movie_id character varying(45) NOT NULL REFERENCES movies (id),
#                 "url" text,
#                 "path" text NOT NULL,
#                 sentiment character varying(20) NOT NULL,
#                 PRIMARY KEY ("id")
#             ); '''

#     # Execute a command: this creates a new table
#     cursor.execute(create_table_query0)
#     cursor.execute(create_table_query1)
#     cursor.execute(create_table_query2)
#     cursor.execute(create_table_query3)
#     connection.commit()
#     print("Table created successfully in PostgreSQL ")

# except (Exception, Error) as error:
#     print("Error while connecting to PostgreSQL", error)
# finally:
#     if (connection):
#         cursor.close()
#         connection.close()
#         print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',) 

Table created successfully in PostgreSQL 
PostgreSQL connection is closed


In [24]:
connection = psycopg2.connect(user=user,
                                  password=password,
                                  host=host,
                                  port=port,
                                  database=database)

# Create a cursor to perform database operations
cursor = connection.cursor()

PostgreSQL server information


In [ ]:
def sortKeyFunc(s):
    return int((os.path.basename(s)[:-4]).split('_')[0])

reviews_dirs = []
reviews_dirs.append(sorted(glob.glob("aclImdb/train/neg/*"),key=sortKeyFunc))
reviews_dirs.append(sorted(glob.glob("aclImdb/train/pos/*"),key=sortKeyFunc))
reviews_dirs.append(sorted(glob.glob("aclImdb/test/neg/*"),key=sortKeyFunc))
reviews_dirs.append(sorted(glob.glob("aclImdb/test/pos/*"),key=sortKeyFunc))

movies_dirs = [
    "aclImdb/train/urls_neg.txt",
    "aclImdb/train/urls_pos.txt",
    "aclImdb/test/urls_neg.txt",
    "aclImdb/test/urls_pos.txt"
]
m = -1
for movie_url in movies_dirs:
    m += 1
    # open the file which has the urls 
    with open(movies_dirs[m]) as file: 
        lines = file.readlines()
        i = 0
        # Loop over the lines and get the review of this url
        prev = ""
        for line in lines:
            # Split the usrl to get the movie id
            tokens = line.split('/')
            url = "/".join(tokens[:-1])
            movie_id = tokens[-2]
            # If the movie already scraped in the last step
            if prev != movie_id:
                # Read the page as html
                page = requests.get(url)
                soup = BeautifulSoup(page.content, 'html.parser')
                try:
                    title = soup.find_all('h1')[0].text
                except:
                    title = "no title"
                
                # select the movie from the DB corresponding to the movie id
                select_query = f"SELECT id FROM movies WHERE id = '{movie_id}'"
                cursor.execute(select_query)
                rows = cursor.fetchall()
                # if there is no record for this movie make a new one
                if len(rows) == 0:
                    insert_query = 'INSERT INTO movies (id, title, url) VALUES'+ f"(%s,%s,%s);"
                    cursor.execute(insert_query,(movie_id, title, url))
                else:
                    print('movie already exist', i)
            # open the file corresponding to the url and read the review
            with open(reviews_dirs[m][i]) as f:
                review_text = f.readline()
            # insert this review to the DB
            insert_query = 'INSERT INTO reviews (review, movie_id, path, sentiment) VALUES'+ f"(%s,%s,%s,%s);"
            cursor.execute(insert_query,(review_text, movie_id, reviews_dirs[m][i], movies_dirs[m][-7:-4]))
            prev = movie_id
            i+=1
    
connection.commit()

In [ ]:
connection.close()